In [19]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [7]:
data = pd.read_csv('Customer_Segmentation_data.csv', encoding = "ISO-8859-1")

In [8]:
print(data.shape)
data.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [10]:
data.InvoiceDate.dtype

dtype('O')

In [11]:
#convert InvoiceDate colm to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data.InvoiceDate.dtype

dtype('<M8[ns]')

In [13]:
data[data.CustomerID.isna()].shape

(135080, 8)

In [16]:
data.dropna(inplace=True)

In [17]:
data[data.CustomerID.isna()].shape

(0, 8)

In [18]:
# for monetary value we need  a total price
data['TOTALPRICE'] = data['UnitPrice'] * data['Quantity']

In [21]:
# Get the recent last date, when transaction was made
print(data['InvoiceDate'].max() )
recent_transaction_date = data['InvoiceDate'].max() + timedelta(days=1)
recent_transaction_date

2011-12-09 12:50:00


Timestamp('2011-12-10 12:50:00')

In [22]:
# Calculate RFM for each customer

rfm = data.groupby('CustomerID').agg({'InvoiceDate':lambda x:(recent_transaction_date - x.max()).days,
                                     'InvoiceNo':'count',
                                     'TOTALPRICE':'sum'})

In [24]:
rfm.head()

,InvoiceDate,InvoiceNo,TOTALPRICE
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [27]:
rfm.rename(columns={'InvoiceDate':'Recency', 'InvoiceNo':'Frequency', 'TOTALPRICE':'Monetary'}, inplace=True)
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [34]:
recency_labels = range(4, 0, -1)
recency_groups = pd.qcut(rfm['Recency'], q=4, labels=recency_labels)

frequency_labels = range(1, 5)
frequency_groups = pd.qcut(rfm['Frequency'], q=4, labels=frequency_labels)

monti_labels = range(1, 5)
monti_groups = pd.qcut(rfm['Monetary'], q=4, labels=recency_labels)

In [35]:
rfm = rfm.assign(R=recency_groups.values, F=frequency_groups.values, M=monti_groups.values )
rfm.head()

,Recency,Frequency,Monetary,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,4
12347.0,2,182,4310.00,4,4,1
12348.0,75,31,1797.24,2,2,1
12349.0,19,73,1757.55,3,3,1
12350.0,310,17,334.40,1,1,3


In [36]:
rfm['avg'] = rfm[['R','F','M']].sum(axis=1)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,avg
CustomerID,,,,,,,
12346.0,326,2,0.00,1,1,4,6.0
12347.0,2,182,4310.00,4,4,1,9.0
12348.0,75,31,1797.24,2,2,1,5.0
12349.0,19,73,1757.55,3,3,1,7.0
12350.0,310,17,334.40,1,1,3,5.0


In [39]:
rfm.avg.nunique()

10

In [40]:
def assign_segment(val):
    if val<=4:
        return 'Needs Attention'
    elif val==5:
        return 'Signin Required'
    elif val==6:
        return 'Promising'
    elif val==7 or val==8:
        return 'Potential'
    elif val==9:
        return 'Loyal'
    elif val==10:
        return 'Can\'t loose them'
rfm['Segment'] = rfm['avg'].apply(assign_segment)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,avg,Segment
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,4,6.0,Promising
12347.0,2,182,4310.00,4,4,1,9.0,Loyal
12348.0,75,31,1797.24,2,2,1,5.0,Signin Required
12349.0,19,73,1757.55,3,3,1,7.0,Potential
12350.0,310,17,334.40,1,1,3,5.0,Signin Required
